#### Terminal Access

In [3]:
from src import *

def main():
    display_welcome_message()
    while True:
        request = input("Enter your request: ")
        out = chat_with_llama(request)
        if out is not None:
            print("Task completed successfully!")
        if not prompt_for_next_action():
            break

In [12]:
if __name__ == "__main__":
    main()

Welcome to the Operating System Assistant!
This assistant helps you perform various tasks related to the operating system.

User message: write snake program
File Writer Tool
Writing content to file: snake.py
Content successfully written to file: snake.py
Executing Python script: snake.py
Error executing Python script:
  File "/Users/varsha.singh/surfer/snake.py", line 46
    print('Game Over!')</p>
                        ^
SyntaxError: invalid syntax

Fixing errors and rerunning the script...
Error executing Python script after fixing the errors:
  File "/Users/varsha.singh/surfer/snake.py", line 46
    print('Game Over!')</p>
                        ^
SyntaxError: invalid syntax

Command Tool
Executing command: python snake.py
Error executing command:   File "/Users/varsha.singh/surfer/snake.py", line 46
    print('Game Over!')</p>
                        ^
SyntaxError: invalid syntax

Task completed successfully!
Exiting...


In [13]:
if __name__ == "__main__":
    main()

Welcome to the Operating System Assistant!
This assistant helps you perform various tasks related to the operating system.

User message: write python program to create pyramid
File Writer Tool
Writing content to file: pyramid.py
Content successfully written to file: pyramid.py
Executing Python script: pyramid.py
Python script executed successfully:
    *
   ***
  *****
 *******
*********

Command Tool
Executing command: python pyramid.py
Command output:
    *
   ***
  *****
 *******
*********

Task completed successfully!

User message: what is the time now
Command Tool
Executing command: date
Command output:
Mon May  6 00:23:07 +08 2024

Task completed successfully!
Exiting...


In [25]:
from pptx import Presentation
import openai
from pptx.util import Pt
from pptx.enum.text import PP_ALIGN

client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

def get_gpt_response(prompt):
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
            {"role": "system", "content": f"Prompt: {prompt}\n"},
            {"role": "user", "content": prompt}
        ],
    )
    chat_completion = response.choices[0].message
    return chat_completion.content.strip()

def generate_slides(topic):
    prs = Presentation()
    
    default_font = "Montserrat"

    title_slide_layout = prs.slide_layouts[0]
    slide = prs.slides.add_slide(title_slide_layout)
    title = slide.shapes.title
    title.text = f"Presentation on {topic}"
    title.text_frame.paragraphs[0].font.bold = True
    title.text_frame.paragraphs[0].font.size = Pt(44)
    title.text_frame.paragraphs[0].font.name = default_font

    # Initialize context
    context = ""

    prompts = [
        "Introduction to " + topic,
        "Problem Statement",
        "Solution Approach",
        "Results",
        "Conclusion"
    ]

    for prompt in prompts:
        slide_layout = prs.slide_layouts[1]  
        slide = prs.slides.add_slide(slide_layout)
        title = slide.shapes.title
        title.text = prompt
        title.text_frame.paragraphs[0].font.size = Pt(22)
        title.text_frame.paragraphs[0].font.name = default_font

        content_box = slide.placeholders[1]
        content = get_gpt_response(context + prompt)
        content_box.text = content
        title.text_frame.paragraphs[0].font.size = Pt(14)
        content_box.text_frame.paragraphs[0].font.name = default_font
        content_box.text_frame.text_anchor = PP_ALIGN.LEFT

        context += content
        #print(context)

    prs.save(f'{topic}_presentation.pptx')
    print(f"Presentation on '{topic}' has been generated.")



topic = input("Enter the topic for your presentation: ")

generate_slides(topic)


The Northern Lights, also known as the Aurora Borealis, are a natural light display that is predominantly seen in the high-latitude regions around the Arctic and Antarctic. These spectacular lights are a result of collisions between electrically charged particles from the sun that enter the earth's atmosphere and collide with gases such as oxygen and nitrogen.

The phenomenon is named after Aurora, the Roman goddess of dawn, and Boreas, the Greek name for the north wind. The lights appear in many colors, although pale green and pink are the most common. Shades of red, yellow, green, blue, and violet can also be seen. The variety of colors depends on the type of gas particles that are colliding. The lights are seen above the magnetic poles of the northern and southern hemispheres.

The best time to view the Northern Lights is during the winter months when the nights are longer and the skies are darker, typically from September to April. They are often visible in the magnetic polar regio

In [10]:
request = "write a python file to find square root"
out = chat_with_llama(request)
#print(out)


# has_bug = "Error" in out
# if has_bug:
#     print("Please debug")
# else:
#     print("think about the next step to complete {initial_goal}")
#     .....


User message: write a python file to find square root
File Writer Tool
Writing content to file: sqrt.py
Content successfully written to file: sqrt.py
Executing Python script: sqrt.py
Python script executed successfully:

Command Tool
Executing command: python sqrt.py
Command output:

